In [ ]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
import dv_lib.dv_importers as dv_importers

import numpy as np
import pandas as pd
import open3d as o3d
import pyvista as pv
import matplotlib.pyplot as plt
import plotly.express as px
import cv2 as cv


#### Load raw volume

In [ ]:
#load raw volume
scan = dv_importers.dv_importFromRaw(".\\Contour\\volume_export_768x768x1280_uint8_t.raw", T=True)

In [ ]:
# preprocess
thinned = scan.copy()
for i in range(thinned.shape[2]):
    slice = scan[:,:,i]
    slice = cv.fastNlMeansDenoising(slice,h=20,templateWindowSize=10,searchWindowSize=21)
    #slice = cv.threshold(slice,80,255,cv.THRESH_BINARY)[1]
    thinned[:,:,i] = cv.ximgproc.thinning(slice)

In [ ]:
def plot_slices(scan, n_slices=5):
    slices = scan[:,:,[z for z in range(0, scan.shape[2], int(scan.shape[2]/n_slices))]].T
    # fig = px.imshow(slices, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"), template='plotly_dark')
    fig = px.imshow(slices, facet_col=0, binary_string=True, facet_col_wrap=5, template='plotly_dark')
    #fig.update_layout(showlegend=False)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    return fig

plot_slices(scan, 5).show()
plot_slices(thinned, 5).show()

In [ ]:
def plot_album(scan, n_slices=4):
    fig, axes = plt.subplots(nrows=1, ncols=n_slices, figsize=(50,50))

    slices = scan[:,:,[z for z in range(0, scan.shape[2], int(scan.shape[2]/n_slices))]]
    for slice, ax in zip(slices.T, axes.ravel()):
        ax.imshow(slice[:,:])
        ax.axis('off')
    fig.tight_layout()

plot_album(scan, n_slices=5)
plot_album(thinned, n_slices=5)

#### Load raw volume into a unifor 3D grid with pyvista

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
grid.dimensions = np.array(thinned.shape) + 1

# Edit the spatial reference
grid.origin = (0, 0, 0)  # The bottom left corner of the data set
grid.spacing = (0.49479, 0.49479, 0.3125)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.cell_data["values"] = thinned.flatten(order="F")  # Flatten the array!

# Now plot the grid!
grid.plot(volume=True)

#### Filtering all the non zero values and saving its corresponding xyz from the filtered grid points

In [ ]:
# delete values bellow 200
threshed = grid.threshold(100)
threshed.plot()

cells = threshed.cell_data['values']
points = threshed.points

# create surface from volume (not good)
# surf = threshed.extract_surface()
# surf.plot(show_scalar_bar=False)
# surf.save('py_mesh.ply')

#### Computing radial normals

In [ ]:
def radial_norms(points, inward=False):
    i = -1 if inward else 1
    xmean = points[:, 0].mean()
    ymean = points[:, 1].mean()
    zmean = points[:, 2].mean()
    dist = points[:, 2].max() - points[:, 2].min()
    normals = []
    for x, y, z in points:
        azimuth = np.arctan2((x-xmean), (y-ymean))
        dip = np.arcsinh((z-zmean)/dist)
        normals.append([i*np.sin(azimuth), i*np.cos(azimuth), dip])
    return np.array(normals)

normals = radial_norms(points)

#### Creating point cloud object to input PSR

In [ ]:
pcd = o3d.geometry.PointCloud() # create an empty poun cloud object

pcd.points = o3d.utility.Vector3dVector(points) # feed the xyz coordinates

pcd.colors = o3d.utility.Vector3dVector(plt.get_cmap('hsv')((points[:, 2] - points[:, 2].min()) / (points[:, 2].max() - points[:, 2].min()))[:, :3]) # feed the rgb colors

pcd.normals = o3d.utility.Vector3dVector(normals) # feed the normal vectors

o3d.visualization.draw_geometries([pcd], point_show_normal=True) # display the pcd

#### Recalculating normlas (optional)

In [ ]:
pcd.estimate_normals()
#pcd.orient_normals_consistent_tangent_plane(10)
#pcd.orient_normals_towards_camera_location(np.array([0, 0, 0]))

o3d.visualization.draw_geometries([pcd], point_show_normal=True) # display the pcd

#### Runing PSR

In [ ]:
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, scale=1.1, linear_fit=False) # running PSR algorithm
mesh.compute_triangle_normals()
#o3d.visualization.draw_geometries([mesh], mesh_show_back_face=False) # display the mesh

#### Removing low density vertices

In [ ]:
#  visualize mesh vertrex densities
densities = np.asarray(densities)
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(plt.get_cmap('plasma')((densities - densities.min()) / (densities.max() - densities.min()))[:, :3])
#o3d.visualization.draw_geometries([density_mesh], mesh_show_back_face=True)

In [ ]:
# removing low density vertices
vertices_to_remove = densities < np.quantile(densities, 0.05)
mesh.remove_vertices_by_mask(vertices_to_remove)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
o3d.io.write_triangle_mesh("03d_mesh.ply", mesh, compressed=True, write_vertex_normals=False)

#### Saving surface mesh file

In [ ]:
data = pv.read('03d_mesh.ply')
data.save('py_mesh.ply')

#### The same but using gareth's contour ply file as input

In [ ]:
#load countour file as a pcd directly instead of mesh
#mesh = o3d.io.read_triangle_mesh(".\\Contour\\Contour.ply")
pcd = o3d.io.read_point_cloud(".\\Contour\\Contour.ply")

In [ ]:
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

In [ ]:
pcd.get_center()

In [ ]:
def radial_norms(points, inward=False):
    i = -1 if inward else 1
    xmean = points[:, 0].mean()
    ymean = points[:, 1].mean()
    zmean = points[:, 2].mean()
    dist = points[:, 2].max() - points[:, 2].min()
    normals = []
    for x, y, z in points:
        azimuth = np.arctan2((x-xmean), (y-ymean))
        dip = np.arcsinh((z-zmean)/dist)
        normals.append([i*np.sin(azimuth), i*np.cos(azimuth), dip])
    return np.array(normals)

points = np.asarray(pcd.points)
normals = radial_norms(points, inward=False)

In [ ]:
pcd = o3d.geometry.PointCloud() # create an empty poun cloud object

pcd.points = o3d.utility.Vector3dVector(points) # feed the xyz coordinates

pcd.colors = o3d.utility.Vector3dVector(plt.get_cmap('hsv')((points[:, 2] - points[:, 2].min()) / (points[:, 2].max() - points[:, 2].min()))[:, :3]) # feed the rgb colors

pcd.normals = o3d.utility.Vector3dVector(normals) # feed the normal vectors

o3d.visualization.draw_geometries([pcd], point_show_normal=True) # display the pcd

In [ ]:
pcd.normalize_normals()
o3d.visualization.draw_geometries([pcd], point_show_normal=True) # display the pcd

#### Runing PSR

In [ ]:
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, scale=1.1, linear_fit=False) # running PSR algorithm
mesh.compute_triangle_normals()
#o3d.visualization.draw_geometries([mesh], mesh_show_back_face=False) # display the mesh

#  visualize mesh vertrex densities
densities = np.asarray(densities)
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = mesh.vertices
density_mesh.triangles = mesh.triangles
density_mesh.triangle_normals = mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(plt.get_cmap('plasma')((densities - densities.min()) / (densities.max() - densities.min()))[:, :3])
# o3d.visualization.draw_geometries([density_mesh], mesh_show_back_face=True)

# removing low density vertices
vertices_to_remove = densities < np.quantile(densities, 0.05)
mesh.remove_vertices_by_mask(vertices_to_remove)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
o3d.io.write_triangle_mesh("03d_mesh.ply", mesh, compressed=True, write_vertex_normals=True)

In [ ]:
mesh_out = mesh.filter_smooth_laplacian(number_of_iterations=100)
mesh_out.compute_triangle_normals()
o3d.visualization.draw_geometries([mesh_out], mesh_show_back_face=True)

In [ ]:
normals2 = np.asarray(pcd.normals)

In [ ]:
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

### Offtopic

In [ ]:
#xy = [[xy, xy] for xy in range(0, 768*0.49479, 0.49479)]

xyz= []
for z in np.arange(0, 1280*0.3125, 0.3125):
    for xy in np.arange(0, 768*0.49479, 0.49479):
        xyz.append([xy, xy, z])
xyz = np.array(xyz)

test = np.array([[x, y, z] for x in np.arange(0, 768*0.49479, 0.49479) for y in np.arange(0, 768*0.49479, 0.49479) for z in np.arange(0, 1280*0.3125, 0.3125)])

In [ ]:
# examples/Python/Basic/working_with_numpy.py

import copy
import numpy as np
import open3d as o3dye

# generate some neat n times 3 matrix using a variant of sync function
x = np.linspace(-3, 3, 401)
mesh_x, mesh_y = np.meshgrid(x, x)
z = np.sinc((np.power(mesh_x, 2) + np.power(mesh_y, 2)))
z_norm = (z - z.min()) / (z.max() - z.min())
xyz = np.zeros((np.size(mesh_x), 3))
xyz[:, 0] = np.reshape(mesh_x, -1)
xyz[:, 1] = np.reshape(mesh_y, -1)
xyz[:, 2] = np.reshape(z_norm, -1)
# print('xyz')
# print(xyz)

# # Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(xyz)
# #o3d.io.write_point_cloud("../../TestData/sync.ply", pcd)

# # Load saved point cloud and visualize it
# #pcd_load = o3d.io.read_point_cloud("../../TestData/sync.ply")
# o3d.visualization.draw_geometries([pcd_load])

# # convert Open3D.o3d.geometry.PointCloud to numpy array
# xyz_load = np.asarray(pcd_load.points)
# print('xyz_load')
# print(xyz_load)

# # save z_norm as an image (change [0,1] range to [0,255] range with uint8 type)
# img = o3d.geometry.Image((z_norm * 255).astype(np.uint8))
# o3d.io.write_image("../../TestData/sync.png", img)
# o3d.visualization.draw_geometries([img])